In [1]:
from PIL import Image
import os
import shutil
import numpy as np
# import cv2
import re
import pickle
import random
import time
# from tqdm import tqdm
import pandas as pd 
from glob import glob
from xml.etree.ElementTree import parse
# from pascal_voc_writer import Writer
import matplotlib.pyplot as plt

import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.extend(['./detr/'])

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

import sys
sys.path.extend(['./util/'])
from data_parser import *
from image_prepro import *
# from torchvision import transforms

from visualize import *

In [2]:
# !pip3 install opencv-python
# !pip3 install tqdm
# !pip3 install sklearn
# !pip3 install albumentations
# !pip3 install pillow
# !pip3 install pandas
# !pip3 install torch
# !pip3 install torchvision
# !pip3 install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !conda install -y pytorch torchvision cudatoolkit=10.2 -c pytorch

In [3]:
# !git clone https://github.com/facebookresearch/detr.git

In [2]:
src_path = '/home/beomgon/Object_Detection/Dataset'
df = pd.DataFrame(columns=['label', 'org_label', 'bbox', 'xmin','ymin','xmax','ymax','width','height', 'file_name', 'path'])
# df = pd.DataFrame(columns=['class','bbox', 'path'])
data_list = []
for dir_name in ['SS', 'SS2']:
    dpath = os.path.join(src_path, dir_name)
#     print(dpath)
    dir_lists = os.listdir(dpath) 
#     print(dir_lists)
    
    for dir_list in dir_lists :
        dpath2 = os.path.join(dpath, dir_list)
        xml_list = [os.path.join(dpath2, d) for d in os.listdir(dpath2) if d.endswith(".xml")]
#         print(xml_list)
        for xml_path in xml_list :
            parser = XMLParser(xml_path)
            if (parser.height, parser.width) not in parser.rejection_size:  
                filename = parser.file_name
                for obj in parser.objects :
    #                     obj_class = obj[0]
    #                     if obj_class in rej_table :
    #                         continue
    #                     print(obj)
                    img_path = '/'.join((xml_path.split('/')[:-1])) + '/' + filename
    #                 img_path = re.sub('xml', 'jpg', xml_path)
    # #                     print(img_path)
                    obj.append(parser.width)
                    obj.append(parser.height)
                    obj.append(parser.file_name)
                    obj.append(img_path)
                    data_list.append(obj)
    # #                     print(obj)


In [3]:
df = pd.DataFrame(data_list,columns=['label', 'org_label', 'bbox', 'xmin','ymin','xmax','ymax','width','height', 'file_name', 'path'])
df = df[['label', 'org_label', 'width','height', 'bbox', 'xmin','ymin','xmax','ymax', 'file_name', 'path']]
# print(df.head())
# df = df.reset_index(drop=True)
# df.drop(columns = ['index'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,label,org_label,width,height,bbox,xmin,ymin,xmax,ymax,file_name,path
0,Carcinoma,Suamous cell carcinoma,4032,1960,"[2061, 1051, 2178, 1175]",2061,1051,2178,1175,20200623_100015.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,HSIL,HSIL,4032,1960,"[1945, 870, 2024, 941]",1945,870,2024,941,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,HSIL,HSIL,4032,1960,"[2400, 947, 2484, 1020]",2400,947,2484,1020,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,HSIL,HSIL,4032,1960,"[2209, 845, 2307, 912]",2209,845,2307,912,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,HSIL,HSIL,4032,1960,"[2056, 811, 2144, 866]",2056,811,2144,866,20200420_100401.jpg,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [6]:
# #     width in albumentation is x asix, so ymax - xmax
# # df = df[['label', 'org_label', 'width','height', 'bbox', 'ymin', 'xmin','ymax','xmax', 'file_name', 'path']]
# # df.drop(columns=['bbox'], inplace=True)

# df.head()

In [7]:
# set(df.width)
# set(df.height)

In [8]:
# import operator

# width = []
# height = []
# xmax_list = list(df.xmax)
# xmin_list = list(df.xmin.apply(lambda x : -1*x))
# ymax_list = list(df.ymax)
# ymin_list = list(df.ymin.apply(lambda x : -1*x))

# height = list(map(operator.add, xmax_list, xmin_list))
# width = list(map(operator.add, ymax_list, ymin_list))
# df['box_width'] = width
# df['box_height'] = height
# df['x_min'] = list(df.ymin)
# df['y_min'] = list(df.xmin)
# df.drop(columns=['bbox', 'xmax', 'ymax', 'xmin', 'ymin'], inplace=True)
# df.head()

In [9]:
# # drop NA cell
# print((len(df)))
# df.label = df.label.apply(lambda x : np.nan if x == '' else x)
# df.dropna(subset=['label'], inplace=True)
# df.reset_index(drop=True, inplace=True)
# print((len(df)))

In [10]:
# def label_to_id(x) :
#     x = str(x)
#     if x == 'ASCUS' :
#         return 0
#     elif x == 'Carcinoma' :
#         return 1
#     elif x == 'HSIL' :
#         return 2
#     elif x == 'LSIL' :
#         return 3
#     elif x == 'Normal' :
#         return 4
    
# df['label_id'] = df.label.apply(lambda x : label_to_id(x))
# df.head()

In [11]:
# df = df[['file_name', 'width', 'height', 'x_min', 'y_min', 'box_width', 'box_height', 'org_label', 'label', 'label_id',  'path']]
# df.head()

In [12]:
# df.to_csv('df.csv', index=None)

In [13]:
df = pd.read_csv('df.csv')
df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
0,20200623_100015.jpg,4032,1960,1051,2061,124,117,Suamous cell carcinoma,Carcinoma,1,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,20200420_100401.jpg,4032,1960,870,1945,71,79,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,20200420_100401.jpg,4032,1960,947,2400,73,84,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,20200420_100401.jpg,4032,1960,845,2209,67,98,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,20200420_100401.jpg,4032,1960,811,2056,55,88,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [14]:
# images = list(range(len(df)))
# # images

In [15]:
image_list = []
def get_data() :
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['x_min','y_min','box_width','box_height']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values

In [16]:
# df_data = df.groupby('file_name')
# images = list(set(df.file_name))
df_data = df.groupby('path')
images = list(set(df.path))
len(images)
# len(df_data)
# len(df_data.groups)
# df_data.get_group('20200420_100401.jpg')

18824

In [17]:
def get_data(img_id):
    if img_id not in df_data.groups:
        return dict(image_id=img_id, source='', boxes=list())
    
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['x_min','y_min','box_width','box_height']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values
    return dict(image_id = img_id, labels=labels, label_id=label_id, boxes = boxes)

image_list = [get_data(img_id) for img_id in images]

In [18]:
# BOX_COLOR = (255, 0, 0) # Red
# TEXT_COLOR = (255, 255, 255) # White

# def visualize_bbox(img, bbox, w=0, h=0, color=BOX_COLOR, thickness=2, IsNormalize=True):
#     """Visualizes a single bounding box on the image"""
#     print('IsNormalize', IsNormalize)
#     if IsNormalize  :
#         x, y, width, height = bbox

#         x_min = int((x - width)*w)
#         y_min = int((y - height)*h)
#         x_max = int((x + width)*w)
#         y_max = int((y + height)*h)   
        
#     else :
# #         print(type(bbox))
# #         bbox[:,2:] /= 2
# #         bbox[:,:2] += bbox[:,2:] 
#         x_min, y_min, x_max, y_max = list(map(int, bbox))
#     print(x_min, y_min, x_max, y_max)

#     img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
# #     ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
# #     cv2.rectangle(img, (y_min - int(1.3 * text_height), x_min ), (y_min, x_min + text_width ), BOX_COLOR, -1)
# #     cv2.putText(
# #         img,
# #         text=class_name,
# #         org=(x_min, y_min - int(0.3 * text_height)),
# #         fontFace=cv2.FONT_HERSHEY_SIMPLEX,
# #         fontScale=0.35, 
# #         color=TEXT_COLOR, 
# #         lineType=cv2.LINE_AA,
# #     )
#     return img

# def visualize(image, bboxes, w=0, h=0, category_ids=None, category_id_to_name=None, IsNormalize=True ):
#     if type(image) is np.ndarray :
#         img = image.copy()
#     else : # pytorch tensor
#         img = image.clone().data.cpu().numpy()
#     print(type(img))
# #     print(IsNormalize)
# #     for bbox, category_id in zip(bboxes, category_ids):
# #         class_name = category_id_to_name[category_id]
# #         print(class_name)
#     for bbox in bboxes :
# #         print(bbox)
#         img = visualize_bbox(img, bbox, w, h, IsNormalize=IsNormalize )
#     plt.figure(figsize=(10, 10))
#     plt.axis('off')
#     plt.imshow(img)    

In [19]:
# # print(image_list[0])
# Number = 282
# path = image_list[Number]['image_id'] 
# boxes = image_list[Number]['boxes']
# print(path)
# new_bboxes = []
# for i in boxes :
#     xmin, ymin, box_width, box_height = i
#     new_bboxes.append([xmin, ymin, xmin + box_width, ymin + box_height])
# # print(new_bboxes)
# # print(path)
# image = cv2.imread(path)
# # print(type(image))
# w, h, _ = image.shape
# # print(w, h)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = cv2.flip(image, 1)

# visualize(image, new_bboxes, w, h, IsNormalize=False)

In [20]:
# boxes = A.augmentations.bbox_utils.normalize_bboxes([[634, 934, 128, 75]],rows=1200,cols=1000)
# boxes

In [21]:
# def get_train_transforms():
#     return A.Compose([
#         A.CenterCrop(1600,1600, True,1),
#         A.RandomCrop(height=800, width=800, p=0.5),
#         A.Resize(height=512, width=512, p=1),

#         A.OneOf([
#         A.HorizontalFlip(p=0.9),
#         A.VerticalFlip(p=0.9),
#         A.RandomRotate90(p=0.9),        
#         ], p=1),
#         A.GaussNoise(p=0.5),   
#             A.Normalize(max_pixel_value=1),
#             #A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
#             ToTensorV2(p=1.0)
#         ], 
#         p=1.0,         
#         bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
#         )

# def get_valid_transforms():
#     return A.Compose([A.Resize(height=512, width=512, p=1.0),
#                       A.Normalize(max_pixel_value=1),
#                       ToTensorV2(p=1.0),
#                       ], 
#                       p=1.0, 
#                       bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
#                       )

In [22]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1560,1560, True,1),
#     A.RandomCrop(height=800, width=800, p=0.5),
#     A.Resize(height=512, width=512, p=1),
    A.Resize(height=256, width=256, p=1),
    
#     A.OneOf([
#     A.HorizontalFlip(p=0.9),
#     A.VerticalFlip(p=0.9),
#     A.RandomRotate90(p=0.9),        
#     ], p=1),
# #     A.GaussNoise(p=0.5),
#     A.OneOf([
#     A.OpticalDistortion(p=0.7),
#     A.GaussNoise(p=0.7)        
#     ], p=1),
#     A.RandomBrightnessContrast(p=0.2),
#     A.Normalize(max_pixel_value=1),
#     ToTensorV2(p=1.0),
    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [23]:

# Number = 160
# path = image_list[Number]['image_id'] 
# boxes = image_list[Number]['boxes']
# labels = image_list[Number]['labels']
# image = cv2.imread(path)
# w, h, _ = image.shape
# # print(w, h)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = cv2.flip(image, 1)

# transformed = transforms(image=image, bboxes=boxes, labels=labels)
# transformed_image = transformed['image']
# transformed_bboxes = transformed['bboxes']
# transformed_class_labels = transformed['labels']
# # print(type(transformed_bboxes[0]))
# # print(boxes)
# # print(transformed_bboxes)
# new_bboxes = []
# for i in transformed_bboxes :
#     xmin, ymin, box_width, box_height = i
#     new_bboxes.append([xmin, ymin, xmin + box_width, ymin + box_height])
# print(transformed_class_labels)
# visualize(transformed_image, new_bboxes,  w, h, IsNormalize=False)

In [44]:
# DIR_TRAIN = '../input/global-wheat-detection/train'
class WheatDataset(Dataset):
    def __init__(self,image_list,transforms=None):
        self.images = image_list
        self.transforms = transforms
        self.img_ids = {x['image_id']:i for i,x in enumerate(image_list)}
        
    def get_indices(self,img_ids):
        return [self.img_ids[x] for x in img_ids]
        
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self,index):
        record = self.images[index]
        image_id = record['image_id']
#         print(image_id)

        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = cv2.flip(image, 1)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = record['boxes'] 
#         print('boxes', boxes)
        
#         labels =  np.zeros(len(boxes), dtype=np.int32)
        labels = record['label_id']
#         labels = np.array(labels, dtype=np.int32)
#         print('label', type(labels))
#         print(labels)
#         print(image.shape)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        _,h,w = image.shape
        h = w = 256
#         print(image.shape)
#         print('h', h)
#         print('w',w)
#         print(h, w)
#         print(sample['bboxes'])
#         print(type(sample['bboxes'][0]))
#         for i in sample['bboxes'][0] :
#             if abs(i) > h :
#                 print('h', h)
#                 print('bbox error')
#                 print(sample['bboxes'])
# #                 break
        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
#         print('boxes', boxes)
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] /= 2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
        target['image_id'] = torch.tensor([index])
        
        return image, target, image_id

In [45]:
train_ds = WheatDataset(image_list,transforms)
# valid_ds = WheatDataset(image_list,get_valid_transforms())

In [46]:
# train_ds[0][1]

In [47]:

# image, target, image_id = train_ds[280]
# w, h, _ = image.shape
# # print(w, h)
# boxes = target['boxes']
# # print('boxes', boxes)
# boxes = A.augmentations.bbox_utils.denormalize_bboxes(boxes,rows=h,cols=w)
# # print('boxes', boxes)
# new_bboxes = []
# for i in boxes :
#     x, y, w, h = i
#     new_bboxes.append([x-w, y-h, x+w, y+h])
# # print(new_bboxes)    
# # print('new_bboxes', new_bboxes)
# # path = image_id
# # image = cv2.imread(path)
# # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# # image = cv2.flip(image, 1)
# visualize(image, new_bboxes, w, h, IsNormalize=False)


In [48]:
# train_ds[1]

In [49]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [50]:
train_data_loader = DataLoader(
#     torch.utils.data.Subset(train_ds,train_indexes),
    train_ds,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
#     collate_fn=None
)

In [51]:
# # def DETRModel(num_classes,model_name=model_name):
# #     model = torch.hub.load('facebookresearch/detr', model_name, pretrained=False, num_classes=num_classes)
# #     def parameter_groups(self):
# #         return { 'backbone': [p for n,p in self.named_parameters()
# #                               if ('backbone' in n) and p.requires_grad],
# #                  'transformer': [p for n,p in self.named_parameters() 
# #                                  if (('transformer' in n) or ('input_proj' in n)) and p.requires_grad],
# #                  'embed': [p for n,p in self.named_parameters()
# #                                  if (('class_embed' in n) or ('bbox_embed' in n) or ('query_embed' in n)) 
# #                            and p.requires_grad]}
# #     setattr(type(model),'parameter_groups',parameter_groups)
# #     return model

# class DETRModel(nn.Module):
#     def __init__(self,num_classes=1):
#         super(DETRModel,self).__init__()
#         self.num_classes = num_classes
        
#         self.model = torch.hub.load('facebookresearch/detr', model_name, pretrained=True)
        
#         self.out = nn.Linear(in_features=self.model.class_embed.out_features,out_features=num_classes+1)
        
#     def forward(self,images):
#         print(type(images))
#         d = self.model(images)
#         d['pred_logits'] = self.out(d['pred_logits'])
#         return d
    
#     def parameter_groups(self):
#         return { 
#             'backbone': [p for n,p in self.model.named_parameters()
#                               if ('backbone' in n) and p.requires_grad],
#             'transformer': [p for n,p in self.model.named_parameters() 
#                                  if (('transformer' in n) or ('input_proj' in n)) and p.requires_grad],
#             'embed': [p for n,p in self.model.named_parameters()
#                                  if (('class_embed' in n) or ('bbox_embed' in n) or ('query_embed' in n)) 
#                            and p.requires_grad],
#             'final': self.out.parameters()
#             }

In [52]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [53]:
n_folds = 5
seed = 42
null_class_coef = 0.5
# num_classes = 1
num_queries = 100
# BATCH_SIZE = 2
LR = 5e-5
lr_dict = {'backbone':0.1,'transformer':1,'embed':1,'final': 5}
EPOCHS = 2
max_norm = 0
model_name = 'detr_resnet50'

In [54]:
# model = DETRModel()
# model.parameter_groups().keys()
#type(model)

In [55]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

# matcher = HungarianMatcher(cost_giou=2,cost_class=1,cost_bbox=5)
matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 5 , 'loss_giou': 2}

losses = ['labels', 'boxes', 'cardinality']

In [56]:
num_classes = 5
criterion = SetCriterion(num_classes, 
                         matcher, weight_dict, 
                         eos_coef = null_class_coef, 
                         losses=losses)

In [57]:
model = DETRModel(num_classes=5, num_queries=num_queries)
# model.summary()

Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master


In [58]:
# device = torch.device('cuda')
# model.to(device)

In [59]:
device = torch.device('cuda')
criterion = criterion.to(device)
model = DETRModel(num_classes=5, num_queries=num_queries)
model = model.to(device)
model.train()
criterion.train()
tk0 = tqdm(train_data_loader, total=len(train_data_loader),leave=False)
log = None
device='cuda'
for step, (images, targets, image_ids) in enumerate(tk0):
#     print(targets)
#     print(targets)
#     print(images[0].shape)
#     print(images[1].shape)
#     print(len(images))
    print(image_ids)
    images = list(image.to(device) for image in images)
#     print(images[0])
#     print(type(images))
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    output = model(images)
#     print(type(output))
#     print(output.keys())
#     print(type(output['pred_boxes']))
#     print(output['pred_boxes'].size())
#     print(output['pred_logits'].size())
#     print(type(targets))
#     print(targets[0]['boxes'].size())
#     print(targets[0].keys())
#     print(targets[0]['boxes'].size())
#     print(type(targets[0]['labels']))
#     print((targets[0]['labels']))
# #     print((targets[1]['labels']))
#     print(len(targets))

    loss_dict = criterion(output, targets)
# #     dfsdfs
#     if log is None:
#         log = {k:AverageMeter() for k in loss_dict}
#         log['total_loss'] = AverageMeter()
#         log['avg_prec'] = AverageMeter()

#     weight_dict = criterion.weight_dict

#     total_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

#     optimizer.zero_grad()

#     total_loss.backward()

# #     if max_norm > 0:
# #         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

#     optimizer.step()    
    
    

Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master
  0%|          | 5/18824 [00:00<26:09, 11.99it/s]  

('/home/beomgon/Object_Detection/Dataset/SS2/05(201020)/20201020_091655.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/06/20200615_140127.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS2/01(200806)/20200826_130028.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/07/20200709_141445.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/04/20200515_140446.png',)
('/home/beomgon/Object_Detection/Dataset/SS/08(200904)/20200724_134428.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/04/20200513_134029.png',)


  0%|          | 12/18824 [00:00<14:21, 21.84it/s]

('/home/beomgon/Object_Detection/Dataset/SS/01/20200207_134232.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS2/01(200806)/20200810_094336.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/08(200904)/20200724_115113.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/08(200824)/20200715_141147(0).jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/04/20200513_113514.png',)
('/home/beomgon/Object_Detection/Dataset/SS2/03(200908-10-normal)/20200909_085856.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/04/20200518_105512.png',)


('/home/beomgon/Object_Detection/Dataset/SS2/05(201019)/20201019_142735.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/04/20200513_094438.png',)
('/home/beomgon/Object_Detection/Dataset/SS2/05(201020)/20201020_140507.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS2/06(201026)/20201026_131318.jpg',)
('/home/beomgon/Object_Detection/Dataset/SS/08(200827)/20200720_160051.jpg',)


ValueError: Caught ValueError in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-44-96e7d45c91df>", line 41, in __getitem__
    sample = self.transforms(**sample)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/albumentations/core/composition.py", line 180, in __call__
    p.preprocess(data)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/albumentations/core/ut

In [43]:

df[df['path'] == '/home/beomgon/Object_Detection/Dataset/SS/01/20200131_104506.jpg']

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
41338,20200131_104506.jpg,4032,1960,243,2056,73,82,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/01/2...
41339,20200131_104506.jpg,4032,1960,380,2069,89,98,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/01/2...
41340,20200131_104506.jpg,4032,1960,1398,2151,111,92,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/01/2...
41341,20200131_104506.jpg,4032,1960,724,2380,111,76,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/01/2...
41342,20200131_104506.jpg,4032,1960,624,2125,63,116,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/01/2...


In [60]:
df[df['path'] == '/home/beomgon/Object_Detection/Dataset/SS/08(200827)/20200720_160051.jpg']

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
23597,20200720_160051.jpg,4032,1960,841,2055,193,123,Squamous cell carcinoma,Carcinoma,1,/home/beomgon/Object_Detection/Dataset/SS/08(2...


In [41]:
df[df['path'] == '/home/beomgon/Object_Detection/Dataset/SS/04/20200513_141113.png']

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
15134,20200513_141113.png,1632,1560,703,805,106,94,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
15135,20200513_141113.png,1632,1560,638,1120,66,52,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...


In [ ]:
out_prob = torch.tensor([[0.2369, 0.1790, 0.0908, 0.2360, 0.2573],
        [0.1606, 0.1870, 0.0675, 0.3291, 0.2558],
        [0.2082, 0.1901, 0.0820, 0.3253, 0.1944],
        [0.1048, 0.2228, 0.0688, 0.2919, 0.3117],
        [0.1060, 0.1564, 0.0623, 0.3033, 0.3719],
        [0.2059, 0.1989, 0.0792, 0.3204, 0.1955],
        [0.1749, 0.2239, 0.0955, 0.3518, 0.1540],
        [0.1904, 0.2015, 0.0887, 0.3251, 0.1943],
        [0.2164, 0.1581, 0.0985, 0.2328, 0.2941],
        [0.1147, 0.2238, 0.0754, 0.2984, 0.2877],
        [0.1516, 0.2839, 0.0860, 0.3140, 0.1646],
        [0.1962, 0.2339, 0.0897, 0.2788, 0.2014],
        [0.2014, 0.2448, 0.0873, 0.2672, 0.1993],
        [0.1794, 0.2555, 0.0884, 0.3367, 0.1401],
        [0.1673, 0.2419, 0.0735, 0.3543, 0.1630],
        [0.2082, 0.1528, 0.0913, 0.2985, 0.2492],
        [0.1115, 0.1988, 0.0663, 0.2971, 0.3262]])

In [ ]:
tgt_id = torch.tensor([2, 2, 2])

In [ ]:
out_prob[:,tgt_id].size()

In [ ]:
out_prob[:, [1, 1]]

In [ ]:
!which python
import sys
sys.executable
sys.path

In [ ]:
import torch
from torch import nn
m = nn.Conv1d(16,33,3, stride=2)
m = m.to('cuda')
input = torch.randn(20,16,50)
input = input.to('cuda')
output = m(input)

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.tensor([1.0, 2.0])

In [ ]:
import torch
from torch import nn
torch.tensor([1.0, 2.0]).cuda()

In [ ]:
256*6